In [ ]:
import torch
import numpy as  np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

# PyTorch in Venice

*most of this talk is stolen from [Stefan Sotte](https://github.com/sotte/pytorch_tutorial)*


## Outline

1. PyTorch Basics

    1.1. 1D Waves in PyTorch
    
    1.2. Example Networks


2. Predicting Storm Surge in Venice

    2.1. Datasets
    
    2.2. Dataloaders
    
    2.3. Network Models

# 1. PyTorch Basics

It is super easy to install (including CUDA!)

    conda install pytorch -c pytorch

### What is it?

* A Deep Learing Library similar to Tensorflow
* Numpy on the GPU

In [ ]:
import tensorflow as tf

x = tf.random_uniform(shape=[5, 5])
y = tf.random_uniform(shape=[5, 5])
s = x + y
s

In [ ]:
with tf.Session() as sess:
    s_ = sess.run(s)
s_

### Dynamic Computational Graph
![graph](dynamic_graph.gif)

In [ ]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
x + y

## 1.2. 1D waves in PyTorch

Use it like Numpy!

In [ ]:
x = torch.arange(0., 10., 0.1)
y = torch.sin(x)

plt.plot(x.numpy(), y.numpy())
plt.show()

In [ ]:
def step_1dwave(eta, vel, dt, dx, g, H):
    vel[0] += dt * ( -g * (eta[1] - eta[-1]) / (2 * dx))
    eta[0] += dt * ( -H * (vel[1] - vel[-1]) / (2 * dx))
    vel[-1] += dt * ( -g * (eta[0] - eta[-2]) / (2 * dx))
    eta[-1] += dt * ( -H * (vel[0] - vel[-2]) / (2 * dx))

    for j in range(1, vel.size(0)-1):
        vel[j] += dt * ( -g * (eta[j+1] - eta[j-1]) / (2 * dx))
        eta[j] += dt * ( -H * (vel[j+1] - vel[j-1]) / (2 * dx))
    return eta, vel


def simulate_1dwave(gpts, width, steps):
    g = 9.81
    H = 1.
    L = width
    x = torch.linspace(0., width, gpts)
    dx = x[1] - x[0]

    vel = torch.zeros(gpts, dtype=torch.float)
    eta = torch.sin(2*np.pi*x/L)
    eta = torch.exp(-(50-x)**2/10**2)
    
    ETA = torch.zeros((steps, gpts))
    VEL = torch.zeros_like(ETA)
    for i in tqdm(range(steps)):
        eta, vel = step_1dwave(eta, vel, dt, dx, g, H)
        ETA[i] = eta
        VEL[i] = vel
    return ETA.numpy(), VEL.numpy()

In [ ]:
gpts = 200
width = 200.
steps = 200
dt = 0.2

ETA, VEL = simulate_1dwave(gpts, width, steps)

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

def animate_line(lines):
    # First set up the figure, the axis, and the plot element we want to animate
    fig, ax = plt.subplots()

    ax.set_xlim(( 0, lines.shape[1]))
    ax.set_ylim((lines.min()-0.1, lines.max()+1.0))

    line, = ax.plot([], [], lw=2)

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return (line,)

    # animation function. This is called sequentially
    def animate(i):
        x = np.arange(lines.shape[1])
        line.set_data(x, lines[i])
        return (line,)

    # call the animator. blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=len(lines), interval=20, blit=True)
    return anim

In [ ]:
anim = animate_line(ETA)
HTML(anim.to_html5_video())

### Utilize the GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
x = torch.rand(5, 5).to(device)
y = torch.rand(5, 5).to(device)
s = x + y
s.to('cpu')

## 1.2. Neural Networks

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from collections import OrderedDict

# Simple sequential model
layers = OrderedDict([
    ('conv1', nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5)),
    ('relu1', nn.ReLU()),
    ('conv2', nn.Conv2d(20,64,5)),
    ('relu2', nn.ReLU())
])
model = nn.Sequential(layers)
model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # you could write ifs / loops here !
        return x


net = Net()
net

### Random number generator in the forward pass!

In [ ]:
class Seq2SeqModel(nn.Module):
    def __init__(self, p_teacher_forcing: float):
        self.p_teacher_forcing = p_teacher_forcing
        # ...

    def forward(self, X, y):
        # ... some calculation
        current_word = torch.zeros(...)
        result = []
        for i in range(self.sentence_length):
            # ... some calculation with current_word
            result.append(output)
            current_word = torch.argmax(output)

            # teacher forcing
            if self.p_teacher_forcing > random.random():
                current_word = y[i]

        return torch.stack(result)